In [ ]:
%reload_ext sql

In [ ]:
%load_ext sql
%sql sqlite:////content/chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @/content/chinook.db'

# Часть 1

In [ ]:
%%sql
SELECT * FROM tracks LIMIT 10

 * sqlite:////content/chinook.db
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99


## Задание 1

Вывести 20 самых больших по размеру треков жанра ROCK и формата MPEG (используйте команду LIMIT)

In [ ]:
%%sql
SELECT * FROM genres
WHERE genres.Name = "Rock";

SELECT *
FROM media_types 
WHERE Name = "MPEG audio file";

 * sqlite:////content/chinook.db
Done.
Done.


MediaTypeId,Name
1,MPEG audio file


In [ ]:
%%sql
SELECT Name FROM tracks
WHERE GenreId = 1 AND MediaTypeId = 1
ORDER BY Bytes DESC
LIMIT 20;

 * sqlite:////content/chinook.db
Done.


Name
Dazed And Confused
Space Truckin'
Dazed And Confused
We've Got To Get Together/Jingo
Funky Piano
Going Down / Highway Star
Santana Jam
The Sun Road
Whole Lotta Love
Mistreated (Alternate Version)


## Задание 2
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра ROCK, приобретенные сотрудниками Microsoft

In [ ]:
%%sql
SELECT artists.Name AS "~Группа~"
      , tracks.Name AS "~Песня~"
      , albums.Title AS "~Альбом~"
FROM ((((artists INNER JOIN albums ON artists.ArtistId = albums.ArtistId) 
INNER JOIN tracks ON albums.AlbumId = tracks.AlbumId) 
INNER JOIN invoice_items ON tracks.TrackId = invoice_items.TrackId)
INNER JOIN invoices ON invoice_items.InvoiceId = invoices.InvoiceId)
INNER JOIN CUSTOMERS ON invoices.CustomerId = CUSTOMERS.CustomerId
WHERE tracks.GenreId = 1 AND CUSTOMERS.Company = 'Microsoft Corporation' 

 * sqlite:////content/chinook.db
Done.


~Группа~,~Песня~,~Альбом~
Foo Fighters,No Way Back,In Your Honor [Disc 1]
Foo Fighters,Still,In Your Honor [Disc 2]
Foo Fighters,Razor,In Your Honor [Disc 2]
Foo Fighters,My Hero,The Colour And The Shape
Guns N' Roses,Double Talkin' Jive,Use Your Illusion I
Guns N' Roses,The Garden,Use Your Illusion I
Guns N' Roses,Don't Damn Me,Use Your Illusion I
Guns N' Roses,Dead Horse,Use Your Illusion I


## Задание 3
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых все треки стоят больше 1,5$ и для которых в наборе есть хотя бы один трек


In [ ]:
%%sql
SELECT genres.Name AS "Жанр"
      , media_types.Name AS "Тип медиа"
      , ROUND(AVG(tracks.UnitPrice), 2) AS "Ср.цена"
      , COUNT(tracks.Name) AS "Кол-во"
FROM tracks, genres, media_types
WHERE tracks.UnitPrice > 1.5 AND genres.GenreId = tracks.GenreId
      AND media_types.MediaTypeId = tracks.MediaTypeId
GROUP BY tracks.GenreId
HAVING COUNT(tracks.TrackId) >= 1


 * sqlite:////content/chinook.db
Done.


Жанр,Тип медиа,Ср.цена,Кол-во
Science Fiction,Protected MPEG-4 video file,1.99,13
TV Shows,Protected MPEG-4 video file,1.99,93
Sci Fi & Fantasy,Protected MPEG-4 video file,1.99,26
Drama,Protected MPEG-4 video file,1.99,64
Comedy,Protected MPEG-4 video file,1.99,17


## Задание 4
Вывести компании, сделавшие максимальное и минимальное число заказов. Необходимо исключить клиентов без компании (NULL)

In [ ]:
%%sql
SELECT CUSTOMERS.Company AS "Компания"
FROM CUSTOMERS INNER JOIN invoices ON CUSTOMERS.CustomerId = invoices.CustomerId
GROUP BY CUSTOMERS.Company
HAVING COUNT(invoices.InvoiceId) IN (
  (SELECT MAX(max_order)
   FROM (
        SELECT CUSTOMERS.Company, COUNT(invoices.InvoiceId) AS max_order
        FROM CUSTOMERS INNER JOIN invoices ON CUSTOMERS.CustomerId = invoices.CustomerId
        GROUP BY CUSTOMERS.Company
        )
  ),
  (SELECT MIN(min_order)
     FROM (
        SELECT CUSTOMERS.Company, COUNT(invoices.InvoiceId) AS min_order
        FROM CUSTOMERS INNER JOIN invoices ON CUSTOMERS.CustomerId = invoices.CustomerId
        GROUP BY CUSTOMERS.Company
    )
  )
 ) AND CUSTOMERS.Company IS NOT NULL

 * sqlite:////content/chinook.db
Done.


Компания
Apple Inc.
Banco do Brasil S.A.
Embraer - Empresa Brasileira de Aeronáutica S.A.
Google Inc.
JetBrains s.r.o.
Microsoft Corporation
Riotur
Rogers Canada
Telus
Woodstock Discos


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру поп-музыки

In [ ]:
%%sql
--Почему-то нелязя испоьзовать функцию IF(условие, истина, ложь)
--Исходя из этого, те компании, которые не попали в список не покупали песни заданного жанра.
SELECT CUSTOMERS.Company AS "Компания"
        , COUNT(tracks.TrackId) AS "Кол-во песен"
FROM (((CUSTOMERS INNER JOIN invoices ON invoices.CustomerId = CUSTOMERS.CustomerId)
      INNER JOIN invoice_items ON invoice_items.InvoiceId = invoices.InvoiceId)
      INNER JOIN tracks ON tracks.TrackId = invoice_items.TrackId)
      INNER JOIN genres ON tracks.GenreId = genres.GenreId
WHERE genres.Name = "Pop"
GROUP BY CUSTOMERS.Company 
HAVING "Компания" != "Null"


 * sqlite:////content/chinook.db
Done.


Компания,Кол-во песен
Embraer - Empresa Brasileira de Aeronáutica S.A.,2
JetBrains s.r.o.,4


* Задание 6*
Для каждого артиста определить все жанры песен, которые у него есть и количество песен по каждому жанру

In [ ]:
%%sql
SELECT artists.Name AS "Артист"
    , genres.Name AS "Жанр"
    , COUNT(tracks.TrackId) AS "Кол-во песен"
FROM ((artists INNER JOIN albums ON artists.ArtistId = albums.ArtistId)
      INNER JOIN tracks ON albums.AlbumId = tracks.AlbumId)
      INNER JOIN genres ON tracks.GenreId = genres.GenreId 
GROUP BY artists.Name, genres.Name
LIMIT 10;

 * sqlite:////content/chinook.db
Done.


Артист,Жанр,Кол-во песен
AC/DC,Rock,18
Aaron Copland & London Symphony Orchestra,Classical,1
Aaron Goldberg,Jazz,1
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,2
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,Classical,1
"Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair",Classical,1
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,1
Accept,Rock,4
Adrian Leaper & Doreen de Feis,Classical,1
Aerosmith,Rock,15


## Задание 7
Вывести средний размер альбома в байтах

In [ ]:
%%sql
SELECT DISTINCT albums.Title AS "Альбом"
      , AVG(tracks.Bytes) AS "Ср.размер"
FROM tracks INNER JOIN albums ON tracks.AlbumId = albums.AlbumId
GROUP BY tracks.AlbumId
LIMIT 5

 * sqlite:////content/chinook.db
Done.


Альбом,Ср.размер
For Those About To Rock We Salute You,7827041.4
Balls to the Wall,5510424.0
Restless and Wild,4871098.0
Let There Be Rock,10029878.0
Big Ones,9618496.866666667


# Часть 2

## Задание 8
Вывести названия треков и их продолжительность, которые были приобретены компаниями, которые обслуживаются сотрудниками, нанятыми после 3 сентября 2002 года

In [ ]:
%%sql 
SELECT tracks.Name
      , Milliseconds 
FROM tracks INNER JOIN (SELECT Company , HireDate
                        FROM CUSTOMERS, employees
                        WHERE SupportRepId = EmployeeId	AND HireDate > '2002-09-03')
LIMIT 10

 * sqlite:////content/chinook.db
Done.


Name,Milliseconds
For Those About To Rock (We Salute You),343719
Balls to the Wall,342562
Fast As a Shark,230619
Restless and Wild,252051
Princess of the Dawn,375418
Put The Finger On You,205662
Let's Get It Up,233926
Inject The Venom,210834
Snowballed,203102
Evil Walks,263497


## Задание 9
Для каждого артиста найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение общей суммы, заработанной артистом по всем другим жанрам к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, и оно является максимальным для артиста, то выбирается жанр раньше по алфавиту

In [ ]:
%%sql
SELECT
Artist_Name
      , Genre_Name
      ,SUM(UnitPrice) AS Total_Cash -- вся прибыль по жанрам
FROM tracks t
LEFT JOIN (SELECT Name AS Genre_Name,* FROM genres ) AS g USING (GenreId )
,(SELECT AlbumID, ArtistId FROM albums ) AS al USING (AlbumID )
,(SELECT Name AS Artist_Name,* FROM artists ) AS ar USING (ArtistId)
, (SELECT TrackId FROM invoice_items) AS ii USING (TrackId )

GROUP BY Artist_Name,Genre_Name
HAVING MAX(UnitPrice)
ORDER BY Total_Cash desc, Artist_Name 

 * sqlite:////content/chinook.db
Done.


Артист,Жанр,Максимальная цена,Соотношение
Audioslave,Alternative,13.86,2.86
Titãs,Alternative & Punk,37.62,8.74
Eric Clapton,Blues,31.68,2.53
Toquinho & Vinícius,Bossa Nova,14.85,1.0
Berliner Philharmoniker & Herbert Von Karajan,Classical,2.97,24.67
The Office,Comedy,33.83,1.0
Lost,Drama,87.56,1.45
Frank Sinatra,Easy Listening,23.76,1.0
O Rappa,Electronica/Dance,16.83,1.76
Iron Maiden,Heavy Metal,27.72,1.0


## Задание 10
Вывести альбом, который является минимальным по объему (в байтах) среди альбомов со стоимостью, большей средней цены за альбом по всем альбомам с таким же жанром (если у альбома песни с нескольким жанрами, то жанром является тот, у которого наибольшее количество песен в альбоме)

In [ ]:
%%sql
SELECT albums.Title AS "Альбом"
        , genres.Name AS "Жанр"
        , MIN(tracks.Bytes) AS "Минимальный объём"
        , SUM(tracks.UnitPrice) AS "Цена за альбом"
FROM (albums INNER JOIN tracks ON albums.AlbumId = tracks.AlbumId)
              INNER JOIN genres ON tracks.GenreId = genres.GenreId
GROUP BY albums.Title

In [ ]:
%%sql 
SELECT "Альбом", MIN("Минимальный объём")
FROM 
  (SELECT albums.Title AS "Альбом"
        , genres.Name AS "Жанр"
        , AVG(tracks.UnitPrice) AS "Ср. цена"
  FROM (albums INNER JOIN tracks ON albums.AlbumId = tracks.AlbumId)
              INNER JOIN genres ON tracks.GenreId = genres.GenreId
  GROUP BY albums.Title) AS tr
  INNER JOIN 
  (SELECT albums.Title AS "Альбом2"
        , genres.Name AS "Жанр"
        , MIN(tracks.Bytes) AS "Минимальный объём"
        , SUM(tracks.UnitPrice) AS "Цена за альбом"
  FROM (albums INNER JOIN tracks ON albums.AlbumId = tracks.AlbumId)
              INNER JOIN genres ON tracks.GenreId = genres.GenreId
  GROUP BY albums.Title) AS rt 
  ON tr."Альбом" = rt."Альбом2"
WHERE "Цена за альбом" > "Ср. цена"
LIMIT 10

 * sqlite:////content/chinook.db
Done.


Альбом,"MIN(""Минимальный объём"")"
O Samba Poconé,38747
